En este archivo notebook implementaremos el método TIRM, guardando la salida en un archivo pickle. Este será el archivo que enviemos al servidor.

In [ ]:
#Cargamos los paquetes necesarios.

import pickle
import bson
import pandas as pd
from bson import ObjectId

In [ ]:
#Cargamos tanto los usuarios como todas las interacciones.

filename = "usuarios.pickle"
usuarios = pd.read_pickle(filename)
#Guardamos todos los ids de los usuarios de la muestra para limpiar las interacciones
idsMuestra = usuarios["_id"].to_list()

#Cargamos los datos de los tweets.
filename = "tweets.pickle"
tweets = pd.read_pickle(filename)

#Solo nos quedamos con las interacciones que se hayan realizado entre un par de 
#usuarios cuyos ids hayamos cogido antes.
tweets = tweets[tweets.user_id.isin(idsMuestra)
                & (tweets.in_reply_to_user_id.isin(idsMuestra) | 
                tweets.retweet_or_quote_user_id.isin(idsMuestra))]

In [ ]:
#Unimos las columnas del conjuto de datos tweets referentes al usuario al que va dirigida la interacción,
#ya que este dato está dividido en dos columnas dependiendo del tipo de interacción.

to_user_id = []

for index, row in tweets.iterrows():
    if row["in_reply_to_user_id"] == None:
        
        to_user_id.append(row["retweet_or_quote_user_id"])
        
    else:
        
        to_user_id.append(row["in_reply_to_user_id"])
        
tweets["to_user_id"] = to_user_id

#Tras esto realizamos el group by

prueba = tweets.groupby(['user_id', 'to_user_id', 'tweet_type'])['to_user_id'].count().reset_index(name="count")

In [ ]:
#Una vez agrupados los datos referentes a las interacciones, vamos a descartar del conjunto de usuarios todos 
#aquellos que no realizen ni reciban ninguna interacción.

aliveUsers = []

for index, row in prueba.iterrows():
    
    if (row['user_id'] in aliveUsers) == False:
        
        aliveUsers.append(row['user_id'])
        
    if (row['to_user_id'] in aliveUsers) == False:
        
        aliveUsers.append(row['to_user_id'])

    
    
usuarios = usuarios[usuarios._id.isin(aliveUsers)]

#Machacamos el conjunto de ids de la muestra de usuarios

idsMuestra = usuarios["_id"].to_list()

In [ ]:
#Creamos un diccionario que relaciona todos los usuarios con todos los usuarios que alguna vez han creado contenido
#refiriendose a este, y asocia el número de cada tipo que ha realizado.
interacciones = {}

for index, row in prueba.iterrows():
    
    #Si son de tipo originales no los tenemos en cuenta.
    if row["tweet_type"] != "original":
    
        #Comprobamos si para este usuario hemos creado ya su hueco en el diccionario, si no se crea.
        if row["to_user_id"] in interacciones:
        
            if row["user_id"] in interacciones[row["to_user_id"]]:
                
                interacciones[row["to_user_id"]][row["user_id"]][row["tweet_type"]] = row["count"]
        
            else:
                
                interacciones[row["to_user_id"]][row["user_id"]] = {"quote" : 0, "retweet" : 0, "reply" : 0}
                
                interacciones[row["to_user_id"]][row["user_id"]][row["tweet_type"]] = row["count"]
                
        
        else:
        
            interacciones[row["to_user_id"]] = {}
            
            interacciones[row["to_user_id"]][row["user_id"]] = {"quote" : 0, "retweet" : 0, "reply" : 0}
                
            interacciones[row["to_user_id"]][row["user_id"]][row["tweet_type"]] = row["count"]

In [ ]:
#Ahora vamos a quedarnos con los conjuntos de seguidores de cada usuario pero de forma que todos los usuarios con los
#que nos quedemos también estén en la red ya que si no, no nos proporciona nada. Para ello definimos una función que
#realiza la intersección de dos conjuntos

def intersectionLists(list1, list2):  

    result = list(set(list1).intersection(list2))
    return result

followers = {}

for index, row in usuarios.iterrows():
    
    followers[row["_id"]] = intersectionLists(idsMuestra, row["followers"])
    
idNodos = idsMuestra

In [ ]:
#Ahora vamos a crear las aristas de cada grafo, para ello definimos un diccionario que contendra todas las aristas
#como clave, y el peso de cada una de ellas como valor.

#GRAFO DE FOCO

aristasGF = {}
beta = 0.7

for user1 in interacciones:
    
    for user2 in interacciones[user1]:
        
        if (interacciones[user1][user2]["reply"] > 0 or interacciones[user1][user2]["retweet"] > 0):
            if user1 != user2:
                arista = "("+str(user1)+","+str(user2)+")"
                aristasGF[arista] = beta *  interacciones[user1][user2]["retweet"] + (1-beta) * interacciones[user1][user2]["reply"]
            
            
#GRAFO DE AUTORIDAD

aristasGA = {}

for user1 in interacciones:
    
    for user2 in interacciones[user1]:
        
        if (interacciones[user1][user2]["quote"] > 0) and user1 != user2:
            
            arista = "("+str(user1)+","+str(user2)+")"
            aristasGA[arista] = interacciones[user1][user2]["quote"]
            
            
#GRAFO DE CENTRALIDAD

aristasGC = {}
gamma = 0.6

for user1 in followers:
    
    for user2 in followers[user1]:
        arista = "("+str(user1)+","+str(user2)+")"
        aristasGC[arista] = gamma * len(followers[user1]) + (1 - gamma) * len(followers[user2])

In [ ]:
#Vamos a guardar los diferentes diccionarios que representan las aristas de cada grafo, junto a la lista de 
#todos los nodos. Con estos datos podremos representar toda la información extraida con Gephi

import pickle

f = open("aristasGF.pkl","wb")
pickle.dump(aristasGF,f)
f.close()

f = open("aristasGA.pkl","wb")
pickle.dump(aristasGA,f)
f.close()

f = open("aristasGC.pkl","wb")
pickle.dump(aristasGC,f)
f.close()

f = open("idNodos.pkl","wb")
pickle.dump(idNodos,f)
f.close()

In [ ]:
#Una vez llegados a este punto, tenemos todos los pesos de aristas para cada uno de los grafos, luego podemos 
#comenzar a calcular los pesos para cada nodo en cada grafo. 

#Vamos a realizar un paso previo y muy importante, este es relacionar cada nodo con sus aristas para cada grafo
#Cada nodo con las aristas que salen de el. De esta forma evitamos repetir el proceso muchas veces. 
#Las claves son los ids de los usuarios pero en string.

def nodoAristas(aristasGrafo):
    
    nodoAristas = {}
    for nodoId in idNodos:
        
        nodoid = str(nodoId)
        nodoAristas[nodoid] = []
        for arista in aristasGrafo:
            
            if ("("+nodoid) in arista:
                
                nodoAristas[nodoid].append(arista)
                
    return nodoAristas

nodoAristasGF = nodoAristas(aristasGF)
nodoAristasGA = nodoAristas(aristasGA)
nodoAristasGC = nodoAristas(aristasGC)

#Pasamos ahora al cálculo del peso NIWT:
#No necesitamos introducir nodos a la función ya que los tres grafos trabajan con el mismo conjunto.
def NIWT(aristasGrafo, alpha, nodoA): 
    
    NIWT = {}
    for nodoId in idNodos:
        
        nodoid = str(nodoId)
        pesosAristas = 0
        
        for key in nodoA[nodoid]:
            
            pesosAristas = pesosAristas + aristasGrafo[key]
        
        NIWT[nodoid] = alpha * len(nodoA[nodoid]) + (1 - alpha) * pesosAristas

    return NIWT


NIWTfoco = NIWT(aristasGF, 0.5, nodoAristasGF)
NIWTautoridad = NIWT(aristasGA, 0.5, nodoAristasGA)
NIWTcentralidad = NIWT(aristasGC, 0.5, nodoAristasGC)

In [ ]:
#Ahora pasamos al cálculo de los pesos en la fase de poda del grafo, NPWT.

def NPWT(aristasGrafo, alpha, niwt, nodoA):
    
    NPWT = {}
    for nodo1 in idNodos:
        
        nodo1id = str(nodo1)
        pesoGrafo = 0
        
        for nodo2 in idNodos:
            
            pesosAristas = 0
            nodo2id = str(nodo2)
            
            if nodo2 != nodo1:
                
                for arista in nodoA[nodo2id]:
                    
                    nodo3id = ((arista.split(","))[1])[:-1]
                    
                    pesosAristas = pesosAristas + aristasGrafo[arista] + niwt[nodo3id]
                
                pesoGrafo = pesoGrafo + alpha * len(nodoA[nodo2id]) + (1 - alpha) * pesosAristas
        
        NPWT[nodo1id] = pesoGrafo
    
    return NPWT 

NPWTfoco = NPWT(aristasGF, 0.5, NIWTfoco, nodoAristasGF)
NPWTautoridad = NPWT(aristasGA, 0.5, NIWTautoridad, nodoAristasGA)
NPWTcentralidad = NPWT(aristasGC, 0.5, NIWTcentralidad, nodoAristasGC)

In [ ]:
#También guardamos estos datos de cada grafo por si los necesitaramos más adelante.

f = open("NPWTfoco.pkl","wb")
pickle.dump(NPWTfoco,f)
f.close()

f = open("NPWTautoridad.pkl","wb")
pickle.dump(NPWTautoridad,f)
f.close()

f = open("NPWTcentralidad.pkl","wb")
pickle.dump(NPWTcentralidad,f)
f.close()

In [ ]:
#Una vez tenemos los resultados de cada nodo, los ordenamos en capas

def capas(NPWT):
    
    capas = {}
    
    for nodo in NPWT:
        
        if (NPWT[nodo] in capas):
            
            capas[NPWT[nodo]].append(nodo)
            
        else:
            
            capas[NPWT[nodo]] = [nodo]
            
    return capas

capasFoco = capas(NPWTfoco)
capasAutoridad = capas(NPWTautoridad)
capasCentralidad = capas(NPWTcentralidad)

#Guardamos la cantidad de capas de cada una de las listas para poder cálcular nuestra medida

ncf = len(capasFoco)
nca = len(capasAutoridad)
ncc = len(capasCentralidad)

In [ ]:
#Faltaría ordenar el diccionario por el valor de sus key, y luego cambiar el nombre de las key por 1,2,3..
#lo que serían las capas respectivas. DESCENDENTE


#Para el grafo de foco
puntuaciones = list(capasFoco.keys())
puntuaciones.sort(reverse = True)

capasFocoOrd = {}
contador = 0

for key in puntuaciones:
    
    capasFocoOrd[contador] = capasFoco[key]
    contador = contador + 1

    
    
    
#Para el grafo de autoridad
puntuaciones = list(capasAutoridad.keys())
puntuaciones.sort(reverse = True)

capasAutoridadOrd = {}
contador = 0

for key in puntuaciones:
    
    capasAutoridadOrd[contador] = capasAutoridad[key]
    contador = contador + 1

    
    
    
#Para el grafo de centralidad
puntuaciones = list(capasCentralidad.keys())
puntuaciones.sort(reverse = True)

capasCentralidadOrd = {}
contador = 0

for key in puntuaciones:
    
    capasCentralidadOrd[contador] = capasCentralidad[key]
    contador = contador + 1

In [ ]:
def userCapa(capasOrd):
    
    userCapa = {}
    
    for capa in capasOrd:
        for user in capasOrd[capa]:
            
            userCapa[user] = capa
    
    return userCapa

userCapaFoco = userCapa(capasFocoOrd)
userCapaAutoridad = userCapa(capasAutoridadOrd)
userCapaCentralidad = userCapa(capasCentralidadOrd)

In [ ]:
#Calculamos la medida con la siguiente función 

import math

def TI(userCapa1, userCapa2, userCapa3, nc1, nc2, nc3):
    
    TI = {}
    
    numberUsers = len(userCapa1)
    digitos = len(str(numberUsers))
    h1 = digitos/2
    h = round(digitos/2)
    if h1 > h:
        h + 1
        
    for user in userCapa1:
        if (userCapa1[user] + userCapa2[user] + userCapa3[user]) != 0:
            TI[user] = ( (math.log(userCapa1[user] + userCapa2[user] + userCapa3[user])) / math.log(nc1 + nc2 + nc3) )**(h)
        else:
            TI[user] = 0
    
    return TI
    

TIFinal = TI(userCapaFoco, userCapaAutoridad, userCapaCentralidad, ncf, nca, ncc)

In [ ]:
#Finalmente, con las puntuaciones conseguidas vamos a ordenarlos otra vez, lo que ya será la puntuación final.
TIFGrupos = {}

for user in TIFinal:
    
    if TIFinal[user] in TIFGrupos:
        TIFGrupos[TIFinal[user]].append(user)
    else:
        TIFGrupos[TIFinal[user]] = [user]

puntuaciones = list(TIFGrupos.keys())
puntuaciones.sort(reverse = True)

puntuacionesOrd = {}

for key in puntuaciones:
    
    puntuacionesOrd[key] = TIFGrupos[key]

In [ ]:
#Vamos a comprobar la separación realizada mediante el Average Shell Load ASL
ALS = 0
for capa in puntuacionesOrd:
    
    ALS = ALS + len(puntuacionesOrd[capa])
    
ALS = ALS/len(puntuacionesOrd)
ALS

In [ ]:
#Guardamos el resultado final que nos ordena todos los nodos según su puntuación final.

f = open("puntuacionesOrd.pkl","wb")
pickle.dump(puntuacionesOrd,f)
f.close()

In [ ]:
#Una vez tenemos todas las soluciones ordenadas en cuanto a la puntuación en la medida, vamos a estudiar las 
#soluciones.

#Vamos a tomar los primeros 20 usuarios en el top, 20 usuarios a mitad de ranking y los 20 últimos y con estos ids
#vamos a estudiar las características que los diferencian.


#Veinte primeros
veinPrim = []

for puntuacion in puntuacionesOrd:
        
    for user in puntuacionesOrd[puntuacion]:
        
        if len(veinPrim) < 20:
            
            veinPrim.append(user)
            
          
        else:
            
            break
            
            
#Veinte del centro
veinMid = []

middle = round(len(puntuacionesOrd)/2)
contador = 0 

for puntuacion in puntuacionesOrd:
    
    contador = contador + 1
    
    if contador >= middle:
        
        for user in puntuacionesOrd[puntuacion]:
            
            if len(veinMid) == 20:
                
                break
                
            veinMid.append(user)
        
    
#Veinte del final

veinFin = []

for puntuacion in reversed(list(puntuacionesOrd.keys())):
        
    for user in puntuacionesOrd[puntuacion]:
        
        if len(veinFin) < 20:
            
            veinFin.append(user)
            
          
        else:
            
            break

In [ ]:
#Ya disponemos de los 60 usuarios que vamos a comparar.

#Para comparar sus posiciones dentro de los diferentes nodos de la red vamos a representar los 3 con gephi.


#Debemos comparar también el resto de datos. Vamos a guardar para cada usuario todos los datos totales 
#que componen sus perfiles.

veinprim = {}
veinmid = {}
veinfin = {}


#Cargamos los datos que vamos a utilizar

filename = "interacciones.pkl"
interacciones = pd.read_pickle(filename)
filename = "followers.pkl"
followers = pd.read_pickle(filename)

for user in veinPrim:
    
    veinprim[user] = {"followers":0, "quotes":0, "retweets":0, "replies":0}
    
    veinprim[user]["followers"] = len(followers[ObjectId(user)])
    
    if ObjectId(user) in interacciones:
        for u in interacciones[ObjectId(user)]:
        
            veinprim[user]["quotes"] = veinprim[user]["quotes"] + interacciones[ObjectId(user)][u]["quote"]
            veinprim[user]["retweets"] = veinprim[user]["retweets"] + interacciones[ObjectId(user)][u]["retweet"]
            veinprim[user]["replies"] = veinprim[user]["replies"] + interacciones[ObjectId(user)][u]["reply"]
        

for user in veinMid:
    
    veinmid[user] = {"followers":0, "quotes":0, "retweets":0, "replies":0}
    
    veinmid[user]["followers"] = len(followers[ObjectId(user)])
    
    if ObjectId(user) in interacciones:
        for u in interacciones[ObjectId(user)]:
        
            veinmid[user]["quotes"] = veinmid[user]["quotes"] + interacciones[ObjectId(user)][u]["quote"]
            veinmid[user]["retweets"] = veinmid[user]["retweets"] + interacciones[ObjectId(user)][u]["retweet"]
            veinmid[user]["replies"] = veinmid[user]["replies"] + interacciones[ObjectId(user)][u]["reply"]
    
for user in veinFin:
    
    veinfin[user] = {"followers":0, "quotes":0, "retweets":0, "replies":0}
    
    veinfin[user]["followers"] = len(followers[ObjectId(user)])
    
    if ObjectId(user) in interacciones:
        for u in interacciones[ObjectId(user)]:
        
            veinfin[user]["quotes"] = veinfin[user]["quotes"] + interacciones[ObjectId(user)][u]["quote"]
            veinfin[user]["retweets"] = veinfin[user]["retweets"] + interacciones[ObjectId(user)][u]["retweet"]
            veinfin[user]["replies"] = veinfin[user]["replies"] + interacciones[ObjectId(user)][u]["reply"]

In [ ]:
#Necesitamos tener los datos sobre los que vamos a hacer plot dentro de listas para que así funcione el código

veinprim_followers = []
veinprim_quotes = []
veinprim_replies = []
veinprim_retweets = []
for user in veinprim:
    veinprim_followers.append(veinprim[user]['followers'])
    veinprim_quotes.append(veinprim[user]['quotes'])
    veinprim_replies.append(veinprim[user]['replies'])
    veinprim_retweets.append(veinprim[user]['retweets'])
    
veinmid_followers = []
veinmid_quotes = []
veinmid_replies = []
veinmid_retweets = []
for user in veinmid:
    veinmid_followers.append(veinmid[user]['followers'])
    veinmid_quotes.append(veinmid[user]['quotes'])
    veinmid_replies.append(veinmid[user]['replies'])
    veinmid_retweets.append(veinmid[user]['retweets'])
    
veinfin_followers = []
veinfin_quotes = []
veinfin_replies = []
veinfin_retweets = []
for user in veinfin:
    veinfin_followers.append(veinfin[user]['followers'])
    veinfin_quotes.append(veinfin[user]['quotes'])
    veinfin_replies.append(veinfin[user]['replies'])
    veinfin_retweets.append(veinfin[user]['retweets'])

In [ ]:
#Pasamos a hacer los plots con todos los datos disponibles.

#Primero con respecto al valor del número de seguidores.

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

def autolabel(rects, xpos='center'):
    """
    Attach a text label above each bar in *rects*, displaying its height.

    *xpos* indicates which side to place the text w.r.t. the center of
    the bar. It can be one of the following {'center', 'right', 'left'}.
    """

    ha = {'center': 'center', 'right': 'left', 'left': 'right'}
    offset = {'center': 0, 'right': 1, 'left': -1}

    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(offset[xpos]*3, 3),  # use 3 points offset
                    textcoords="offset points",  # in both directions
                    ha=ha[xpos], va='bottom')

ind = np.arange(len(veinprim_followers))  # the x locations for the groups
width = 0.1  # the width of the bars

fig, ax = plt.subplots(figsize=(13,5))
rects1 = ax.bar(ind - width, veinprim_followers, width,
                label='veinprim_followers')
rects2 = ax.bar(ind, veinmid_followers, width,
                label='veinmid_followers')
rects3 = ax.bar(ind + width, veinfin_followers, width,
                label='veinfin_followers')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Número de seguidores')
ax.set_title('Seguidores por posición en el ranking')
ax.set_xticks(ind)
#ax.set_xticklabels(('G1', 'G2', 'G3', 'G4', 'G5'))
ax.legend()

autolabel(rects1, "left")
autolabel(rects2, "center")
autolabel(rects3, "right")

fig.tight_layout()

plt.show()

In [ ]:
#Ahora comparamos para el número de menciones

def autolabel(rects, xpos='center'):
    """
    Attach a text label above each bar in *rects*, displaying its height.

    *xpos* indicates which side to place the text w.r.t. the center of
    the bar. It can be one of the following {'center', 'right', 'left'}.
    """

    ha = {'center': 'center', 'right': 'left', 'left': 'right'}
    offset = {'center': 0, 'right': 1, 'left': -1}

    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(offset[xpos]*3, 3),  # use 3 points offset
                    textcoords="offset points",  # in both directions
                    ha=ha[xpos], va='bottom')

ind = np.arange(len(veinprim_quotes))  # the x locations for the groups
width = 0.1  # the width of the bars

fig, ax = plt.subplots(figsize=(13,5))
rects1 = ax.bar(ind - width, veinprim_quotes, width,
                label='veinprim_quotes')
rects2 = ax.bar(ind, veinmid_quotes, width,
                label='veinmid_quotes')
rects3 = ax.bar(ind + width, veinfin_quotes, width,
                label='veinfin_quotes')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Número de menciones')
ax.set_title('menciones por posición en el ranking')
ax.set_xticks(ind)
#ax.set_xticklabels(('G1', 'G2', 'G3', 'G4', 'G5'))
ax.legend()

autolabel(rects1, "left")
autolabel(rects2, "center")
autolabel(rects3, "right")

fig.tight_layout()

plt.show()

In [ ]:
#Número de respuestas a los tweets creados

def autolabel(rects, xpos='center'):
    """
    Attach a text label above each bar in *rects*, displaying its height.

    *xpos* indicates which side to place the text w.r.t. the center of
    the bar. It can be one of the following {'center', 'right', 'left'}.
    """

    ha = {'center': 'center', 'right': 'left', 'left': 'right'}
    offset = {'center': 0, 'right': 1, 'left': -1}

    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(offset[xpos]*3, 3),  # use 3 points offset
                    textcoords="offset points",  # in both directions
                    ha=ha[xpos], va='bottom')

ind = np.arange(len(veinprim_replies))  # the x locations for the groups
width = 0.1  # the width of the bars

fig, ax = plt.subplots(figsize=(13,5))
rects1 = ax.bar(ind - width, veinprim_replies, width,
                label='veinprim_replies')
rects2 = ax.bar(ind, veinmid_replies, width,
                label='veinmid_replies')
rects3 = ax.bar(ind + width, veinfin_replies, width,
                label='veinfin_replies')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Número de respuestas')
ax.set_title('respuestas por posición en el ranking')
ax.set_xticks(ind)
#ax.set_xticklabels(('G1', 'G2', 'G3', 'G4', 'G5'))
ax.legend()

autolabel(rects1, "left")
autolabel(rects2, "center")
autolabel(rects3, "right")

fig.tight_layout()

plt.show()

In [ ]:
#Número de retweets

def autolabel(rects, xpos='center'):
    """
    Attach a text label above each bar in *rects*, displaying its height.

    *xpos* indicates which side to place the text w.r.t. the center of
    the bar. It can be one of the following {'center', 'right', 'left'}.
    """

    ha = {'center': 'center', 'right': 'left', 'left': 'right'}
    offset = {'center': 0, 'right': 1, 'left': -1}

    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(offset[xpos]*3, 3),  # use 3 points offset
                    textcoords="offset points",  # in both directions
                    ha=ha[xpos], va='bottom')

ind = np.arange(len(veinprim_retweets))  # the x locations for the groups
width = 0.1  # the width of the bars

fig, ax = plt.subplots(figsize=(13,5))
rects1 = ax.bar(ind - width, veinprim_retweets, width,
                label='veinprim_retweets')
rects2 = ax.bar(ind, veinmid_retweets, width,
                label='veinmid_retweets')
rects3 = ax.bar(ind + width, veinfin_retweets, width,
                label='veinfin_retweets')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Número de retweets')
ax.set_title('retweets por posición en el ranking')
ax.set_xticks(ind)
#ax.set_xticklabels(('G1', 'G2', 'G3', 'G4', 'G5'))
ax.legend()

autolabel(rects1, "left")
autolabel(rects2, "center")
autolabel(rects3, "right")

fig.tight_layout()

plt.show()

In [ ]:
#A primera vista la diferencia es bastante palpable entre el top y el centro, supongo que si amplio el dataset
#tendremos unos plot más cómodos.

#Ahora vamos a comprobar si alguno de los usuarios que hemos cogido son bots, tomamos como medida para que sean bots
#0.691, basándonos en el estudio de los tutores. De esta forma vamos a ver que puntuación les hemos aplicado
#y sus diferentes valores en cuanto a sus parámetros.

#Cargamos los datos de los usuarios, y tomamos solo un trozo de los datos para no guardarlo todo.

filename = "usuarios.pickle"
idsProbBot = pd.read_pickle(filename)
idsProbBot = idsProbBot.iloc[range(0,20000),]
#Todos los ids de los usuarios de la muestra

idsProbBot = idsProbBot[["_id", "scores_scores_universal"]]

posBots = {}

for index, row in idsProbBot.iterrows():
    
    if row["scores_scores_universal"] >= 0.691:
        
        posBots[row["_id"]] = row["scores_scores_universal"]
        
for user in posBots:
    if user in TIFinal:
    
        posBots[user]["TI"] = TIFinal[user]

In [ ]:
#Vamos a crear los archivos csv para introducir dentro del programa gephi

import csv

#Cargamos todos los ids de los usuarios que estamos usando, los relacionamos con sus medidas
filename = "idNodos.pkl"
idNodos = pd.read_pickle(filename)

with open("nodes.csv", "w", newline="") as f:
    thewriter = csv.writer(f)
    
    #La primera fila será el nombre de las diferentes columnas.
    thewriter.writerow(["ID", "NPWTfoco", "NPWTautoridad", "NPWTcentralidad", "TI"])
    
    #Para cada nodo guardamos los tres valores de las columnas
    for user in idNodos:
        idstr = str(user)
        thewriter.writerow([idstr, NPWTfoco[idstr], NPWTautoridad[idstr], NPWTcentralidad[idstr], TIFinal[idstr]])

In [ ]:
#Vamos a crear ahora el archivo txt referente a las aristas
import csv

with open("edgesFoco.csv", "w", newline="") as f:
    thewriter = csv.writer(f)
    
    #La primera fila será el nombre de las diferentes columnas.
    thewriter.writerow(["Source", "Target", "Type", "Weight"])
    
    #Para cada nodo guardamos los tres valores de las columnas
    for edge in aristasGF:
        e = edge.split(",")
        source = e[0][1:]
        target = e[1][:-1]
        thewriter.writerow([source, target, "Directed", aristasGF[edge]])

In [ ]:
with open("edgesAutoridad.csv", "w", newline="") as f:
    thewriter = csv.writer(f)
    
    #La primera fila será el nombre de las diferentes columnas.
    thewriter.writerow(["Source", "Target", "Type", "Weight"])
    
    #Para cada nodo guardamos los tres valores de las columnas
    for edge in aristasGA:
        e = edge.split(",")
        source = e[0][1:]
        target = e[1][:-1]
        thewriter.writerow([source, target, "Directed", aristasGA[edge]])

In [ ]:
with open("edgesCentralidad.csv", "w", newline="") as f:
    thewriter = csv.writer(f)
    
    #La primera fila será el nombre de las diferentes columnas.
    thewriter.writerow(["Source", "Target", "Type", "Weight"])
    
    #Para cada nodo guardamos los tres valores de las columnas
    for edge in aristasGC:
        e = edge.split(",")
        source = e[0][1:]
        target = e[1][:-1]
        thewriter.writerow([source, target, "Directed", aristasGC[edge]])